In [ ]:
!unzip '/content/drive/MyDrive/data'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import os
from torchvision import transforms,datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
num_epochs = 10
num_classes = 10
learning_rate = 0.001
batch_size = 16
input_size = 32*32
hidden_layers = 100

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset initialization
data_dir = 'data' # Suppose the dataset is stored under this folder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']} # Read train and test sets, respectively.

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,shuffle=True, num_workers=2)for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

class_names = image_datasets['train'].classes

trainLoader = dataloaders['train']
testLoader = dataloaders['test']


In [ ]:
class Net2(nn.Module):
  def __init__(self, classes):
    super(Net2, self).__init__()
    self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
    self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

    self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

    self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    # max pooling (kernel_size, stride)
    self.pool = nn.MaxPool2d(2, 2)

    # fully conected layers:
    self.fc6 = nn.Linear(512, 512)
    self.fc7 = nn.Linear(512, 100)
    self.fc8 = nn.Linear(100, 10)
    self.fc9 = nn.Linear(1000,10)
  def forward(self, x, training=True):
    x = F.relu(self.conv1_1(x))
    x = F.relu(self.conv1_2(x))
    x = self.pool(x)
    x = F.relu(self.conv2_1(x))
    x = F.relu(self.conv2_2(x))
    x = self.pool(x)
    x = F.relu(self.conv3_1(x))
    x = F.relu(self.conv3_2(x))
    x = F.relu(self.conv3_3(x))
    x = self.pool(x)
    x = F.relu(self.conv4_1(x))
    x = F.relu(self.conv4_2(x))
    x = F.relu(self.conv4_3(x))
    x = self.pool(x)
    x = F.relu(self.conv5_1(x))
    x = F.relu(self.conv5_2(x))
    x = F.relu(self.conv5_3(x))
    x = self.pool(x)
    x = x.view(-1, 512)
    x = F.relu(self.fc6(x))
    x = F.dropout(x, 0.5, training=training)
    x = F.relu(self.fc7(x))
    x = F.dropout(x, 0.5, training=training)
    x = self.fc8(x)
    return x
    

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(3, 256, 3),
            nn.PReLU(),
            nn.Conv2d(256, 128, 2),
            nn.PReLU(),
            nn.Conv2d(128, 64, 2),
            nn.PReLU(),
            nn.Conv2d(64, 32, 2),
            nn.PReLU(),
            nn.Conv2d(32, 8, 2),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(8),
            nn.Conv2d(8,8,1),
            nn.Conv2d(8,4,1),
            nn.Conv2d(4,4,1),
            nn.Conv2d(4,2,1),  
            nn.Conv2d(2,2,1),
        )

        self.fc = nn.Sequential(
            nn.Linear(338, 64),
            nn.PReLU(),
            nn.Linear(64,20),
            nn.PReLU(),
            nn.Linear(20, 10),
            
        )
      

    def forward(self, img):
        x = self.convs(img)
       # print(x.shape)
        x = x.view(4,338)
        x = self.fc(x)
        return x



In [ ]:
def train_test(model, criterion, optimizer, scheduler, num_epochs=1500):
  for epoch in range(num_epochs):
    running_loss=0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainLoader,0):
      #print(type(data))
      inputs, labels = data
     # inputs, labels = inputs.cuda(), labels.cuda()
      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      scheduler.step()

      running_loss+=loss.item()
    #print(epoch) 
      
      _, predicted = torch.max(outputs.data,1)
      total+=labels.size(0)
      correct+= (predicted==labels).sum().item()
      if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss /2000 ))
        running_loss = 0.0
        print('Accuracy: ', 100*correct/total)
        correct = 0
        total = 0
       # for i,para in enumerate(model.parameters()):
        #  print(f'{i+1}th parameter tensor:')
          #print(para)
        #  print(para.grad)
        #print()
    #  print("Epoch: {}/{}. step: {}/{}, loss: {:.4f}".format(epoch, num_epochs, i, len(trainLoader), loss.item()))

    correct = 0
    total = 0
    with torch.no_grad():
      for data in testLoader:
        images, labels = data
       # images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+= (predicted==labels).sum().item()
    print('Accuracy: ', 100*correct/total)
    
  return None


In [ ]:
class Net3(nn.Module):
    """
    Input - 1x32x32
    Output - 10
    """
    def __init__(self):
        super(Net3, self).__init__()
        # TODO: Initialize layers
        self.conv= nn.Sequential(
          nn.Conv2d(3,32,3),
          nn.ReLU(),
          nn.Conv2d(32,128,2),
          nn.BatchNorm2d(128),
          nn.MaxPool2d(3, 2),
          nn.ReLU(),
          nn.Flatten()
        )


        flatten_size = self.get_flatten_size([4,3,32,32])

        self.fc = nn.Sequential(
            nn.Linear(flatten_size,32),
            nn.ReLU(),
            nn.Linear(32,10),
            nn.Softmax()
        )
    def get_flatten_size(self,shape):
        dummy_img = torch.zeros(size=shape)
        emb_1 = self.conv(dummy_img)
        #conv_fc = self.nin_block(emb_1)
        return int(np.prod(emb_1.shape[1:]))

    def forward(self, img):

        # TODO: Implement forward pass
        emb_1 = self.conv(img)
        # conv_fc = self.nin_block(emb_1)
        y = self.fc(emb_1)
        return y

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net6(nn.Module): 
    def __init__(self):
        super(Net6, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(9216, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(4,9216)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [ ]:
class Net4(nn.Module):
    def __init__(self, num_classes=10):
        super(Net4, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()

        self.fc = nn.Linear(in_features=16 * 16 * 24, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.relu1(output)

        output = self.conv2(output)
        output = self.relu2(output)

        output = self.pool(output)

        output = self.conv3(output)
        output = self.relu3(output)

        output = self.conv4(output)
        output = self.relu4(output)

        output = output.view(-1, 16 * 16 * 24)

        output = self.fc(output)

        return output

In [ ]:
class Net7(nn.Module):
  def __init__(self):
    super(Net7, self).__init__()
    self.conv1 = nn.Conv2d(3, 128, 3)
    self.conv2 = nn.Conv2d(128,)

In [ ]:
model_ft = Net3() # Model initialization

model_ft = model_ft.to(device) # Move model to cpu

criterion = nn.CrossEntropyLoss() # Loss function initialization

# TODO: Adjust the following hyper-parameters: learning rate, decay strategy, number of training epochs.
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001) # Optimizer initialization
#optimizer_ft = optim.Adam(lr=0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1) # Learning rate decay strategy

train_test(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=1500)

torch.save(model.state_dict(), "/content/drive/MyDrive")